<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo-41/blob/main/Reto_Entrega_2_Equipo41.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portada
---

<br>
<p align=center>Instituto Tecnológico y de Estudios Superiores de Monterrey</p>
<p align=center>Escuela de Ingeniería y Ciencias</p>
<p align=center>
<br>
<img src="https://github.com/PosgradoMNA/actividades-del-projecto-equipo-41/blob/main/tec.png?raw=true" alt="Logo" width="250"/>
</p>
<br>
<p align=center>Maestría en Inteligencia Artificial Aplicada (MNA)</p>
<p align=center>TC4029. Ciencia y Analítica de Datos</p>
<p align=center> Profesor Titular: María de la Paz Rico </p>
<p align=center> <b>Reto Final - Parte II (Modelo)</b> </p>
<br>
<p align=center>Presentan:</p>
<p align=center>A01150742 | Ovalle Alvarado José</p>
<p align=center>A01793023 | Arroyo Chavelas Jorge Luis</p>
<p align=center>18 de Noviembre de 2022</p>
<br>
<br>
<p align=center><small><italic>“Por medio de la presente hacemos constar que el reporte que estamos enviando es de nuestra completa autoría y que no estamos haciendo plagio de ideas o escritos del trabajo de otras personas”</italic></small></p>

---

[GitHub Link](https://github.com/PosgradoMNA/actividades-del-projecto-equipo-41/blob/main/Reto_Entrega_1_Equipo41.ipynb)


# Introducción

Preapremos primero nuestro ambiente de análisis:

In [ ]:
# Algunas instalaciones previas para Colab:
!pip install -q qeds fiona geopandas xgboost gensim folium pyLDAvis descartes

     |████████████████████████████████| 16.7 MB 7.2 MB/s 
     |████████████████████████████████| 1.0 MB 44.2 MB/s 
     |████████████████████████████████| 1.7 MB 22.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 179 kB 66.8 MB/s 
     |████████████████████████████████| 6.3 MB 45.9 MB/s 


In [ ]:
############################################################################################
# Librerias y funciones:
############################################################################################

# Para obtener los datos
import requests
import zipfile
import io
import base64

# Para manipualr datos
import pandas as pd
import numpy as np

# Para visualizaciones:
from tabulate import tabulate
from matplotlib import pyplot as plt
import seaborn as sns

# Para particiones y validaciones
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Para transformaciones:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

# Para pipelines:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Para datos geográficos
from shapely.geometry import Point
import geopandas as gpd
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from scipy.spatial import ConvexHull
from PIL import Image

# Otras utilidades
import functools
import warnings

sns.set_theme(style='white', palette=None)
%matplotlib inline

Obtengamos los datos:

In [ ]:
############################################################################################
# Llamar los datos:
############################################################################################

# Definir la url de donde obtener los datos:
datos_url = 'http://201.116.60.46/Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'

# Obtener la respuesta:
datos_response = requests.get(datos_url, stream=True)

# Tomar respuesta como zip en cache:
datos_zip = zipfile.ZipFile(io.BytesIO(datos_response.content))

# Extraer todos los archivos del zip en nuestro espacio local:
datos_zip.extractall("./")

# Definir el lugar donde guardamos los archivos:
datos_dir = './Datos_de_calidad_del_agua_2020'

# Definir archivos a utilizar
datos_file = '/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv'

# Impotar los datos:
df = pd.read_csv(datos_dir + datos_file, encoding = "cp1252")

# Confirmemos que tenemos el dataset:
print(f'\n Dataframe con {df.shape} dimensiones')


 Dataframe con (1068, 57) dimensiones


In [ ]:
############################################################################################
# Datos de entrenamiento y prueba:
############################################################################################

# Conjunto con features
X = df.drop('SEMAFORO', axis=1)

# Variable objetivo
y = df[['SEMAFORO']]

# Partir los datos en conjuntos de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(f"""
Tamaño de variables de entrenamiento: {X_train.shape} || tamaño de objetivo entrenamiento: {y_train.shape}
Tamaño de variables de prueba: {X_test.shape} || tamaño de objetivo prueba: {y_test.shape}
""")


Tamaño de variables de entrenamiento: (854, 56) || tamaño de objetivo entrenamiento: (854, 1)
Tamaño de variables de prueba: (214, 56) || tamaño de objetivo prueba: (214, 1)

